# LAB 4:  BigQuery ML Model Linear Benchmark.

**Learning Objectives**

1. Create benchmark/baseline model with BQML
1. Evaluate benchmark/baseline model
1. Calculate RMSE of benchmark/baseline model


## Introduction 
In this notebook, we will create a benchmark or baseline model to predict the weight of a baby before it is born.  We will use BigQuery ML to build a linear babyweight prediction model with the base features and no feature engineering, yet.

In this lab we will create a benchmark/baseline model with BQML, evaluate our benchmark/baseline model, and calculate the RMSE of our benchmark/baseline model.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/4_bqml_linear_benchmark_babyweight.ipynb).

## Load necessary libraries

Check that the Google BigQuery library is installed and if not, install it.

In [1]:
!pip freeze | grep google-cloud-bigquery==1.6.1 || pip install google-cloud-bigquery==1.6.1

google-cloud-bigquery==1.6.1


## Verify tables exist

Verify that you previously created the dataset and data tables. If not, go back to lab [2_prepare_babyweight](../solutions/2_prepare_babyweight.ipynb) to create them.


In [2]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_train
LIMIT 0

Empty DataFrame
Columns: [weight_pounds, is_male, mother_age, plurality, gestation_weeks]
Index: []

In [3]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_eval
LIMIT 0

Empty DataFrame
Columns: [weight_pounds, is_male, mother_age, plurality, gestation_weeks]
Index: []

## Benchmark Model:  Create the benchmark/baseline Model

Next, we'll create a linear regression baseline model with no feature engineering.  Recall that a model in BigQuery ML represents what an ML system has learned from the training data.  A baseline model is a solution to a problem without applying any machine learning techniques.

### Create the SQL statement to create the model "Benchmark Model".

When creating a BQML model, you must specify the model type (in our case linear regression) and the input label (weight_pounds).  Note also that we are using the training data table as the data source and we don't need BQML to split the data because we have already split it ourselves.

In [4]:
%%bigquery
CREATE OR REPLACE MODEL
    babyweight.benchmark_model

OPTIONS (
    MODEL_TYPE="LINEAR_REG",
    INPUT_LABEL_COLS=["weight_pounds"],
    DATA_SPLIT_METHOD="NO_SPLIT") AS

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_data_train

Empty DataFrame
Columns: []
Index: []


REMINDER:  The query takes several minutes to complete. After the first iteration is complete, your model (benchmark_model) appears in the navigation panel of the BigQuery web UI. Because the query uses a CREATE MODEL statement to create a model, you do not see query results.

You can observe the model as it's being trained by viewing the Model stats tab in the BigQuery web UI. As soon as the first iteration completes, the tab is updated. The stats continue to update as each iteration completes.

Once the training is done, visit the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) and look at the model that has been trained. Then, come back to this notebook.

## Evaluate the benchmark model
Even though BigQuery can automatically split the data it is given, and training on only a part of the data and using the rest for evaluation, to compare with our custom models later we wanted to decide the split ourselves so that it is completely reproducible.

NOTE: The results are also displayed in the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) under the **Evaluation** tab.

In [5]:
%%bigquery
#Information from model training
SELECT * FROM ML.TRAINING_INFO(MODEL babyweight.benchmark_model)

training_run  iteration      loss eval_loss learning_rate  duration_ms
0             0          0  1.139961      None          None        34025

### Get evaluation statistics for the benchmark_model.

After creating your model, you evaluate the performance of the regressor using the ML.EVALUATE function. The ML.EVALUATE function evaluates the predicted values against the actual data.

In [6]:
%%bigquery
SELECT
    *
FROM
    ML.EVALUATE(MODEL babyweight.benchmark_model,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks
    FROM
        babyweight.babyweight_data_eval
    ))

mean_absolute_error  mean_squared_error  mean_squared_log_error  \
0             0.829811            1.139579                 0.02041   

   median_absolute_error  r2_score  explained_variance  
0               0.674778  0.345083            0.345085

**NOTE:** Because you performed a linear regression, the results include the following columns:

*   mean_absolute_error
*   mean_squared_error
*   mean_squared_log_error
*   median_absolute_error
*   r2_score
*   explained_variance

**Resource** for an explanation of the [Regression Metrics](https://towardsdatascience.com/metrics-to-evaluate-your-machine-learning-algorithm-f10ba6e38234).

**Mean squared error** (MSE) - Measures the difference between the values our model predicted using the test set and the actual values. You can also think of it as the distance between your regression (best fit) line and the predicted values. 

**Root mean squared error** (RMSE) - The primary evaluation metric for this ML problem is the root mean-squared error. RMSE measures the difference between the predictions of a model, and the observed values. A large RMSE is equivalent to a large average error, so smaller values of RMSE are better. One nice property of RMSE is that the error is given in the units being measured, so you can tell very directly how incorrect the model might be on unseen data.

**R2**:  An important metric in the evaluation results is the R2 score. The R2 score is a statistical measure that determines if the linear regression predictions approximate the actual data. Zero (0) indicates that the model explains none of the variability of the response data around the mean.  One (1) indicates that the model explains all the variability of the response data around the mean.

### Write a SQL query to find the RMSE of the evaluation data
Since this is regression, we typically use the RMSE, but natively this is not in the output of our evaluation metrics above. However, we can simply take the SQRT() of the mean squared error of our loss metric from evaluation of the benchmark_model to get RMSE.

In [7]:
%%bigquery
SELECT
    SQRT(mean_squared_error) AS rmse
FROM
    ML.EVALUATE(MODEL babyweight.benchmark_model,
    (
    SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality,
        gestation_weeks
    FROM
        babyweight.babyweight_data_eval
    ))

rmse
0  1.06751

## Lab Summary: 
In this lab we created a benchmark/baseline model with BQML, evaluated our benchmark/baseline model, and calculated the RMSE of our benchmark/baseline model.

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License